<div style="border-style: solid;border-width: 1px;">
<div style="height: 200px; 
            width: 750px; 
            color: #ffffff;
            margin: auto;
	        background-color: #ff6600; 
	        background-image: url(http://icarus.lngs.infn.it/img/n3.jpg); 
            background-repeat: no-repeat;
            background-position: center;">
</div>

<div style="text-align:center">
    <h1><font color="blue"><font size="6">Signal Processing in ICARUS</font></font></h1><br>
    <font color="gray"><font size="3">A jupyter notebook aimed at studying Deconvolution with the ICARUS LAr TPC</font></font><br>
</div>
</div>
<br>
<h3>What this notebook intends to do:</h3>
<ul>
    <li>Understand better the signal processing - deconvolution - in ICARUS</li>
        <ul>
            <li>It simulates waveforms based on delta function charge deposits </li>
            <li>We can overlay these waveforms onto waveforms input from data files </li>
            <li>We can then study the components of the deconvolution</li>
                <ul>
                    <li>Forms of filter functions to use in the deconvolution</li>
                    <li>Optimization of parameters</li>
                    <li>etc.</li>
                </ul>
        </ul>
</ul>

<h3>What is needed to run this notebook:</h3>
<ul>
    <li><p>Necessary python libraries/includes</p>
        <ul>
            <li> matplotlib   - the basic plotting package though note that here we mostly use plot-ly</li>
            <li> numpy        - python array handling and loads of useful functions</li>
            <li> scipy        - where we get the analytic functions we will make heavy use of </li>
            <li> ipywidgets   - useful for making interactive output</li>
            <li> uproot       - see below</li>
            <li> plotly       - the plot-ly graphing package which allows interactive plots</li>
        </ul></li>
    <li>uproot<br>
        In this notebook we use "uproot" to handle the root input. For more information on uproot follow this <a href="https://uproot.readthedocs.io/en/latest/">link</a></li>
</ul>

<br>
Useful references for this notebook:
<ul>
    <li><a href="https://arxiv.org/abs/1802.08709">Ionization Electron Signal Processing in Single Phase LArTPCs I. Algorithm Description and Quantitative Evaluation with MicroBooNE Simulation.</a></li>
    <li><a href="https://arxiv.org/abs/1804.02583">Ionization Electron Signal Processing in Single Phase LArTPCs II. Data/Simulation Comparison and Performance in MicroBooNE.</a></li>
    <ul>
        <li>This is a good reference for the mechanics of the deconvolution process in LArTPCs</li>
    </ul>
    <li>An explanation of the scipy signal filters can be found <a href="https://docs.scipy.org/doc/scipy/reference/tutorial/signal.html">here</a></li>
    <li>A good web reference on Digital Signal Processing can be found <a href="https://dsp-nbsphinx.readthedocs.io/en/nbsphinx-experiment/index.html">here</a></li>
</ul>



<div style="text-align:center;border-style: solid;border-width: 1px;">
    <h2><font color="blue"><font size="5">Start Up Block</font></font></h2><br>
</div>

<ul>
    <li>Import of standard libraries used below</li>
    <li>Set the system path to include looking in the package of noise handling functions/objects</li>
    <li>We define two variables which define the location of the input file</li>
        <ul>
            <li> PATHNAME       - Fully qualified path to the folder containing the input file</li>
            <li> RECOFILENAME   - the fully qualified file name for the input data file</li>
        </ul>
    <li>With these definitions we can open and read the input data file</li>
</ul>



In [ ]:
# Probably most people will simply want "matplotlib inline" but on a mac it is useful to do the other stuff
%matplotlib inline
##matplotlib qt
##matplotlib notebook
##config InlineBackend.figure_format ='retina'
#%pylab

# This allows reading of root files without having to actually use root. Nice!
import uproot

# Useful stuff for plotting
import numpy as np
import plotly.graph_objects as go
import plotly.subplots as subplots
from plotly.colors import DEFAULT_PLOTLY_COLORS

import sys

sigProcPath = "/home/usher/LArTPC/icarus-sigproc-tools"
sys.path.insert(0,sigProcPath)

###############################################################################
# TODO:
# Set the path and name of the data file to be read
PATHNAME       = "/home/usher/LArTPC/ICARUS/workarea/data"
RECOFILENAME   = PATHNAME + "/data_run621_1_EW05_PED_20190926T201608_DECODE.root"
#RECOFILENAME   = PATHNAME + "/data_run618_2_EW01M_PED_20190926T174904_DECODE.root"

<div style="text-align:center;border-style: solid;border-width: 1px;">
    <h2><font color="blue"><font size="5">Open and Read the Input File and Process Waveforms</font></font></h2>
</div>

<ul>
    <li>We define three variables which should be common to all files
        <ul>
            <li> RECOFOLDERNAME - the name of the folder for the particular data set we want to analyze</li>
            <li> DAQNAME.       - currently unused</li>
        </ul></li>
    <li>With these definitions we can open and read the input data file</li>
    <li>Contents of the input file are read into a numpy array</li>
    <li>We then run the noise filtering on these waveforms</li>
</ul>



In [ ]:
from sigproc_tools.sigproc_objects.filterevents import FilterEvents

# Below should be standard for the test data files currently available
RECOFOLDERNAME = "Events" 
DAQNAME        = "raw::RawDigits_daq__TPCANALYSIS."

# Grab the pandas dataframe from the input file for the tree we want to look at
print("Opening file: ",RECOFILENAME)
data_file = uproot.open(RECOFILENAME)

print(data_file.keys(),"\n------\n")

print("Opening the folder contianing the RawDigits information: ",RECOFOLDERNAME)
events_folder = data_file[RECOFOLDERNAME]

print(events_folder)

# Go ahead and filter the events
noiseFilter = FilterEvents(events_folder,DAQNAME)

# What are the basic operating parameters?
nChannelsPerGroup = 64   # <==== We choose this
numChannels       = noiseFilter.rawdigits.numChannels(0)
numGroups         = numChannels // nChannelsPerGroup

numEvents = noiseFilter.filterEvents(nChannelsPerGroup)

print("Noise processing complete with",numEvents)

<div style="text-align:center;border-style: solid;border-width: 1px;">
    <h2><font color="blue"><font size="5">Build Filters and Response Functions</font></font></h2>
</div>

<ul>
    <li>Start by building the Filter Functions</li>
        <ul>
            <li> Single gauss function</li>
            <li> Gaussian * (1 - Gaussian)</li>
            <li> A pseudo (or inspired by) Wiener function </li>
            <li> A filter started with a 1 - gaussian and completed with a pseudo Wiener</li>
            <li> <b>NOTE:</b> The input parameters for the filters can be adjusted in this block </li>
        </ul>
    <li>Used these to then build the response/deconvolution functions</li>
    <li>Make some plots to "see" what is happening</li>
</ul>

<b>Note:</b>
The point of this block is to explore filter functions and note their impact on the deconvolution kernels. From here it can also be seen the biggest problem with the 1D deconvolution procedure for the induciton planes, namely that because the response functions go to zero at low frequency the deconvolution will amplify any low frequency noise it sees. 


In [ ]:
from sigproc_tools.sigproc_objects.fullresponse import FullResponse
from sigproc_tools.sigproc_objects.fieldresponse import FieldResponse
from sigproc_tools.sigproc_objects.electronicsresponse import ElectronicsResponse
from sigproc_tools.sigproc_functions.fakeParticle import genWhiteNoiseWaveform,genSpikeWaveform,createParticleTrajectory
from sigproc_tools.sigproc_objects.filter import *

numTimeBins  = 4096
samplingRate = 0.4

# Here the input parameters are defined. 
# Generally, the gaussians are defined by just their offset (which is almost always set to 0) and their width. The values input are in kHz. 
# The Wiener filter will be defined by its offset (again, mostly 0), its scaling factor (sort of sigma for the Gaussian) and exponent. Obviously,
# the Wiener function will revert to a gaussian if the exponent is 2. 
paramsInd0   = [(0.,3.),(0.,80.)]     #[(0.,4.0), (0.,100.)]
paramsInd1   = [(0.,10.),(0.,90.)]   #[(0.,4.0), (0.,100.)]
paramsCol    = (0.,90.)               #(0.,100.)
wienerInd    = (0.,60.,1.7)
gaussWiener  = [(0.,2.),(0.,100.,3.)]

# Here we build the filter functions 
filterInduction0  = FilterDoubleGauss(numTimeBins,samplingRate,paramsInd0)
filterInduction1  = FilterDoubleGauss(numTimeBins,samplingRate,paramsInd1)
filterWiener      = FilterPseudoWiener(numTimeBins,samplingRate,wienerInd)
filterCollection  = FilterGauss(numTimeBins,samplingRate,paramsCol)
filterGaussWiener = FilterGaussPseudoWiener(numTimeBins,samplingRate,gaussWiener)

# Plot the filter functions so we can compare
filterPlot = go.Figure()

filterPlot.add_traces([
    go.Scatter( 
        line=dict(color=DEFAULT_PLOTLY_COLORS[0], width=1),
        name="Filter Induction0",
        x=filterInduction0.frequencyBins,
        y=filterInduction0.filter.real),
    go.Scatter( 
        line=dict(color=DEFAULT_PLOTLY_COLORS[1], width=1),
        name="Filter Induction1",
        x=filterInduction1.frequencyBins,
        y=filterInduction1.filter.real),
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[2], width=1),
        name="Filter Collection",
        x=filterCollection.frequencyBins,
        y=filterCollection.filter.real),
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[3], width=1),
        name="Filter Wiener",
        x=filterWiener.frequencyBins,
        y=filterWiener.filter.real),
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[4], width=1),
        name="Filter Gauss-Wiener",
        x=filterGaussWiener.frequencyBins,
        y=filterGaussWiener.filter.real)
])

filterPlot.show()


# Set up to read and initialize the response functions for each plane
inputFilePath = "/home/usher/LArTPC/ICARUS/workarea/data/"

TPCresponses = [None,None,None,None]

TPCresponses[3] = FullResponse(inputFilePath,"t600_response_vw02_v0.0.root",filterWiener)
TPCresponses[2] = FullResponse(inputFilePath,"t600_response_vw02_v0.0.root",filterCollection)
TPCresponses[1] = FullResponse(inputFilePath,"t600_response_vw01_v0.0.root",filterInduction1,TPCresponses[2].FieldResponse.normFactor)
TPCresponses[0] = FullResponse(inputFilePath,"t600_response_vw00_v0.0.root",filterInduction0,TPCresponses[2].FieldResponse.normFactor)

# Now plot the response functions
responsePlot = go.Figure()

responsePlot.add_traces([
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[0],width=1),
        name="Response Induction 0",
        x=TPCresponses[0].Filter.frequencyBins,
        y=np.absolute(TPCresponses[0].ResponseFFT)
    ),
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[1],width=1),
        name="Response Induction 1",
        x=TPCresponses[1].Filter.frequencyBins,
        y=np.absolute(TPCresponses[1].ResponseFFT)
    ),
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[2],width=1),
        name="Response Collection",
        x=TPCresponses[2].Filter.frequencyBins,
        y=np.absolute(TPCresponses[2].ResponseFFT)
    )
])

responsePlot.show()

# And now plot the deconvolution kernels
deconvPlot = go.Figure()

deconvPlot.add_traces([
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[0],width=1),
        name="Deconvolution Induction 0",
        x=TPCresponses[0].Filter.frequencyBins,
        y=np.absolute(TPCresponses[0].DeconvolutionFFT)
    ),
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[1],width=1),
        name="Deconvolution Induction 1",
        x=TPCresponses[1].Filter.frequencyBins,
        y=np.absolute(TPCresponses[1].DeconvolutionFFT)
    ),
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[2],width=1),
        name="Deconvolution Collection",
        x=TPCresponses[2].Filter.frequencyBins,
        y=np.absolute(TPCresponses[2].DeconvolutionFFT)
    ),
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[3],width=1),
        name="Deconvolution Collection",
        x=TPCresponses[3].Filter.frequencyBins,
        y=np.absolute(TPCresponses[3].DeconvolutionFFT)
    )
])

deconvPlot.show()

<div style="text-align:center;border-style: solid;border-width: 1px;">
    <h2><font color="blue"><font size="5">Create a Simulated Signal and Overaly on Noise Waveform</font></font></h2>
</div>
<br>
In this block we create a test waveform based on a deposit of a given number of electrons. In this case all the electrons input are "seen" by the wire. 
Once we have that we can overlay on top of a waveform from a test noise run taken this past Fall (2019). We can then look at the deconvolution, in particular to understand the impact of filters, noise, etc., on the charge resolution. 
<br>

  

In [ ]:
# Set up for fitting
from scipy.optimize import curve_fit
import scipy.signal as sig
import math

# Define model function to be used to fit to the data above:
def gauss(x, *p):
    A, mu, sigma = p
    return A*np.exp(-(x-mu)**2/(2.*sigma**2))

# Experiment with creating an event pic
# Start with "just" a charge deposit on a zero waveform
numElectrons    = 15000 # number electrons per mm after recombination
electronicsGain = 67.4
numTicks        = 4096

# Here we define the type of response 0=First Induction, 1=Middle Induction, 2=Collection
responseType = 1

#angleToWire  = 90 # degrees
#slope        = math.tan(math.radians(90-angleToWire)) / 0.213
startTick    = 2000
maxRMS       = 3.

eventNum     = 1
wireNum      = 255

kernel       = 3
halfRange    = 256

tickVals     = np.arange(numTicks)

rawWaveform,spikeWaveform = genSpikeWaveform(TPCresponses[responseType],numElectrons,startTick,numTicks)

medWaveform = np.zeros(len(rawWaveform))
medWaveform[startTick-halfRange:startTick+halfRange] = rawWaveform[startTick-halfRange:startTick+halfRange]

# run  the deconvolution on this waveform
rawWaveformFFT = np.fft.rfft(rawWaveform)

outputWaveformFFT = np.multiply(rawWaveformFFT,TPCresponses[responseType].DeconvolutionFFT)

outputWaveform = np.fft.irfft(outputWaveformFFT)

outputWaveform = np.roll(outputWaveform,int(TPCresponses[responseType].T0Offset/TPCresponses[responseType].TPCTickWidth))

# Now let's try fitting the resulting peak to check our charge resolution
peakValue = outputWaveform[startTick]
peakRange = np.where(outputWaveform > 0.5*peakValue)
print("peakRange:",peakRange,", first/last:",peakRange[0][0],",",peakRange[0][-1])
peakSigma = (peakRange[0][-1]-peakRange[0][0])/2.2
print("peakSigma:",peakSigma)
fitParams = np.array([outputWaveform[startTick],peakSigma,startTick]).astype(np.float64)
fitRange  = (startTick-int(round(3.5*peakSigma)),startTick+int(round(3.5*peakSigma)))
fitXVals  = tickVals[fitRange[0]:fitRange[1]]
fitYVals  = outputWaveform[fitRange[0]:fitRange[1]]

print("fitParams:",fitParams)

# here is where the fit is done
coeff,varMatrix = curve_fit(gauss,fitXVals.astype(np.float64),fitYVals.astype(np.float64),p0=fitParams)

print("Fit coefficients - Pulse Height: ",coeff[0],", mean: ",coeff[1],", sigma: ",coeff[2])

totalCharge = electronicsGain * coeff[0] * coeff[2] * math.sqrt(2.* math.pi)

print("Charge input:",numElectrons,", Charge out:",totalCharge)

fitCurve = gauss(fitXVals,*np.array(coeff))

waveformPlot = go.Figure()

waveformPlot.add_traces([
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[0],width=1),
        name="Input Waveform",
        x=tickVals,
        y=spikeWaveform
    ),
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[1],width=1),
        name="Raw Waveform",
        x=tickVals,
        y=rawWaveform
    ),
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[2],width=1),
        name="Med Filt Waveform",
        x=tickVals,
        y=medWaveform
    ),
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[4],width=1),
        name="Deconvolved Waveform",
        x=tickVals,
        y=outputWaveform
    ),
    go.Scatter(
        line=dict(color='red',dash='dash',width=1),
        name="Fit",
        x=fitXVals,
        y=fitCurve
    )
])

waveformPlot.show()

# Do again with noise
rawWaveWithNoise = rawWaveform + noiseFilter.waveLessCoherentAll[eventNum,wireNum,:]

medWaveWithNoiseIn = np.copy(rawWaveWithNoise)
#medWaveWithNoiseIn[startTick-16:startTick+16] = np.zeros(32)
# Well, the idea would be to see if we can take FFT, do some smoothing, and then get something representing the noise on the wavefrom
#medWaveWithNoiseFFT = np.fft.rfft(medWaveWithNoiseIn)
#medWaveWithNoiseFFT.real = sig.medfilt(medWaveWithNoiseFFT.real,kernel)
#medWaveWithNoiseFFT.imag = sig.medfilt(medWaveWithNoiseFFT.imag,kernel)
#medWaveWithNoise    = np.fft.irfft(medWaveWithNoiseFFT)
#medWaveWithNoise = sig.medfilt(medWaveWithNoiseIn,9)
#rawWaveWithNoise -= medWaveWithNoise

medWaveWithNoise = sig.wiener(medWaveWithNoiseIn,16)
#dWaveWithNoise = sig.gauss_spline(medWaveWithNoiseIn,2)

print(type(medWaveWithNoise),medWaveWithNoise)

rawWaveWithNoiseFFT = np.fft.rfft(medWaveWithNoise)

rawWaveOutputFFT = np.multiply(rawWaveWithNoiseFFT,TPCresponses[responseType].DeconvolutionFFT)

rawWaveOutput = np.fft.irfft(rawWaveOutputFFT)

rawWaveOutput = np.roll(rawWaveOutput,int(TPCresponses[responseType].T0Offset/TPCresponses[responseType].TPCTickWidth))

fitYVals  = rawWaveOutput[fitRange[0]:fitRange[1]]

# here is where the fit is done
coeff,varMatrix = curve_fit(gauss,fitXVals.astype(np.float64),fitYVals.astype(np.float64),p0=coeff)

print("Fit coefficients - Pulse Height: ",coeff[0],", mean: ",coeff[1],", sigma: ",coeff[2])

totalCharge = electronicsGain * coeff[0] * coeff[2] * math.sqrt(2.* math.pi)

print("Charge input:",numElectrons,", Charge out:",totalCharge)

fitCurve = gauss(fitXVals,*np.array(coeff))

# Plot time again
noiseWavePlot = go.Figure()

tickVals = np.arange(numTicks)

noiseWavePlot.add_traces([
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[0],width=1),
        name="Input Waveform",
        x=tickVals,
        y=spikeWaveform
    ),
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[1],width=1),
        name="Raw Waveform",
        x=tickVals,
        y=rawWaveWithNoise
    ),
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[2],width=1),
        name="Med Filt Waveform",
        x=tickVals,
        y=medWaveWithNoise
    ),
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[4],width=1),
        name="Deconvolved Waveform",
        x=tickVals,
        y=rawWaveOutput
    ),
    go.Scatter(
        line=dict(color='red',dash='dash',width=1),
        name="Fit",
        x=fitXVals,
        y=fitCurve
    )
])

noiseWavePlot.show()


rawResponsePlot = go.Figure()

rawResponsePlot.add_traces([
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[0],width=1),
        name="Raw Response No Noise",
        x=TPCresponses[0].Filter.frequencyBins,
        y=np.absolute(rawWaveformFFT)
    ),
    go.Scatter(
        line=dict(color=DEFAULT_PLOTLY_COLORS[1],width=1),
        name="Raw Response w/ Noise",
        x=TPCresponses[1].Filter.frequencyBins,
        y=np.absolute(rawWaveWithNoiseFFT)
    )
])

rawResponsePlot.show()
